In [8]:
##  d+s-d+s layers, Calculate free energy in k-space
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
import time
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from scipy import sparse, integrate
import concurrent.futures
import scipy.sparse.linalg as sla
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D


sigma_0 = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j],[1j,0]])
sigma_z = np.array([[1,0],[0,-1]])

tau_0 = sigma_0
tau_x = sigma_x
tau_y = sigma_y
tau_z = sigma_z

# delta_up = (1-p)*delta_d*(cos(kx*pi)-cos(ky*pi))+p*delta_d*(cos(kx*pi)+cos(ky*pi))
# delta_down = (1-p)*delta_d*(cos(2*theta*pi)*(cos(kx*pi)-cos(ky*pi))-sin(2*theta*pi)*2*sin(kx*pi)*sin(ky*pi))+p*delta_d*(cos(kx*pi)+cos(ky*pi))

def H(kx,ky,t,mu,delta_d,p_up,p_down,phi,theta):
    delta_up = (1-p_up)*delta_d*(cos(kx*pi)-cos(ky*pi))+p_up*delta_d*(cos(kx*pi)+cos(ky*pi))
    delta_down = (1-p_down)*delta_d*(cos(2*theta*pi)*(cos(kx*pi)-cos(ky*pi))-sin(2*theta*pi)*2*sin(kx*pi)*sin(ky*pi))+p_down*delta_d*(cos(kx*pi)+cos(ky*pi))
    
    H = 2*t*(2-cos(kx*pi)-cos(ky*pi))*kron(sigma_0,tau_z)-mu*kron(sigma_0,tau_z)+g*kron(sigma_x,tau_z)+\
    np.array([[0,delta_up*(cos(phi*pi)+1j*sin(phi*pi)),0,0],[delta_up*(cos(phi*pi)-1j*sin(phi*pi)),0,0,0],
             [0,0,0,delta_down],[0,0,delta_down,0]])
    return H

t = 1
mu = 2.5
delta_d = 0.2
theta = 0

N_k = 21
Free_energy = np.zeros((5,51,101))
for m in range(5): ## interlayer coupling strength
    print(m)
    g = 0.1*m/4 + 0.05
    for p in range(51):## pairing function
        p_up = p/50
        p_down = 0
        for i in range(101): ## calculate free energy
#             print(i)
            phi = i/50
            Eig = np.zeros((N_k,N_k,2))
            for j in range(N_k):
                for k in range(N_k):
                    kx = 2*j/(N_k-1)
                    ky = 2*k/(N_k-1)
                    ev, ewf=np.linalg.eigh(H(kx,ky,t,mu,delta_d,p_up,p_down,phi,theta))
                    evs = np.sort(ev)
                    Eig[j,k,:] = evs[0:2]
            Free_energy[m,p,i] = ((Eig.sum(axis=2)).sum(axis=1)).sum(axis=0)

# np.save('C:/Users/dell/Desktop/Data/D-wave pairing/Fe_ds_ds.npy',Free_energy)   

phi_dif = np.linspace(0,2,101)
p_dif = np.linspace(0,1,51)
phi_dif_new = np.linspace(0,2,2001)
Free_energy_new = np.zeros((5,51,2001)) ## tc-p-phi

## 对phi做插值
for i in range(5):
    for j in range(51):
        f = interpolate.interp1d(phi_dif,Free_energy[i,j,:],kind = 'quadratic')   ##cubic, quadratic
        Free_energy_new[i,j,:] = f(phi_dif_new)

## plot free energy
pyplot.figure()
for i in range(51):
    pyplot.plot(phi_dif_new,Free_energy_new[4,i,:]-np.min(Free_energy_new[4,i,:]))
pyplot.xlabel(r'$\phi/\pi$')
pyplot.ylabel(r'E')
        
## 寻找 free energy 最小值点
phi_min = np.zeros((5,51))
for i in range(5):
    for j in range(51):
        min_index = np.argmin(Free_energy_new[i,j,0:1000])
        phi_min[i,j] = (phi_dif_new[0:1000])[min_index]
        
## 画出 phi_min
pyplot.figure()
for i in range(5):
    r = round(0.1*i/4 + 0.05,3)
    pyplot.plot(p_dif,phi_min[i,:],label='$t_c='+str((r))+'$')
pyplot.legend(loc='upper left')
pyplot.xlabel(r'$p_{up}$')
pyplot.ylabel(r'$\phi_{min}/\pi$')
pyplot.show()

Using matplotlib backend: Qt5Agg
0
1
2
3
4


In [19]:
Free_energy =np.load('C:/Users/dell/Desktop/Data/D-wave pairing/Fe_ds_ds.npy') 
print(np.shape(Free_energy))
phi_dif = np.linspace(0,2,101)
p_dif = np.linspace(0,1,51)
phi_dif_new = np.linspace(0,2,2001)
Free_energy_new = np.zeros((5,51,2001)) ## tc-p-phi

## 插值
for i in range(5):
    for j in range(51):
        f = interpolate.interp1d(phi_dif,Free_energy[i,j,:],kind = 'quadratic')   ##cubic, quadratic
        Free_energy_new[i,j,:] = f(phi_dif_new)

## plot free energy
pyplot.figure()
for i in range(51):
    pyplot.plot(phi_dif_new,Free_energy_new[4,i,:]-np.min(Free_energy_new[4,i,:]))
pyplot.xlabel(r'$\phi/\pi$')
pyplot.ylabel(r'E')
        
## 寻找 free energy 最小值点
phi_min = np.zeros((5,51))
for i in range(5):
    for j in range(51):
        min_index = np.argmin(Free_energy_new[i,j,0:1000])
        phi_min[i,j] = (phi_dif_new[0:1000])[min_index]

pyplot.figure()
for i in range(5):
    r = round(0.1*i/4 + 0.05,3)
    pyplot.plot(p_dif,phi_min[i,:],label='$t_c='+str((r))+'$')
pyplot.legend(loc='upper left')
pyplot.xlabel(r'$p_{up}$')
pyplot.ylabel(r'$\phi_{min}/\pi$')
pyplot.show()

(5, 51, 101)
